In [1]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
import matplotlib.pyplot as plt

unable to import 'smart_open.gcs', disabling that module
[nltk_data] Downloading package stopwords to /home/amy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Import from Data Prep Notebook

In [2]:
#with open('../data/rev_train.pkl', 'rb') as f:
#    rev_train = pickle.load(f)
#with open('../data/rev_test.pkl', 'rb') as f:
#    rev_test = pickle.load(f)
#train_df = pd.read_csv('may-to-sept_cleaned.csv')
train_df = pd.read_csv('crawled_83k_cleaned.csv')
print(train_df['anger'])
test_df = pd.read_csv('current-tweets_cleaned.csv')

0        0
1        0
2        0
3        0
4        0
        ..
75369    0
75370    0
75371    0
75372    0
75373    0
Name: anger, Length: 75374, dtype: int64


#### Prep Review Text for LDA - Need to make Bigram Model

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['come','order','try','go','get','make','drink','plate','dish','restaurant','place',
                  'would','really','like','great','service','came','got'])

In [4]:
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [5]:
def strip_newline(series):
    return [review.replace('\n','') for review in series]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [6]:
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

In [7]:
 def lemmatization(texts, allowed_postags=['NOUN','ADV']):
     texts_out = []
     for sent in texts:
         doc = nlp(" ".join(sent)) 
         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
     return texts_out

In [8]:
def get_corpus(df):
    """
    Get Bigram Model, Corpus, id2word mapping
    """
    
    df['text'] = strip_newline(df.text)
    words = list(sent_to_words(df.text))
    words = remove_stopwords(words)
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
    lemma = lemmatization(bigram)
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram

In [9]:
#bigram_train4[0]

In [10]:
train_corpus4, train_id2word4, bigram_train4 = get_corpus(train_df)
bigram_train4[0]

['well', 'marcom', 'pal']

#### Load Data Back in for New Sessions

In [11]:
# with open('train_corpus4.pkl', 'wb') as f:
#     pickle.dump(train_corpus4, f)
# with open('train_id2word4.pkl', 'wb') as f:
#     pickle.dump(train_id2word4, f)
# with open('bigram_train4.pkl', 'wb') as f:
#     pickle.dump(bigram_train4, f)
# # with open('lemma_train.pkl', 'wb') as f:
# #     pickle.dump(lemma_train, f)

In [12]:
# # make sure importing right model
# lda_train4 = gensim.models.ldamulticore.LdaMulticore.load('lda_train4.model')

#### Save Data

In [14]:
corpus_lg = None
#with open('tweets_lg.pkl', 'rb') as f:
with open('83k_tweets_lg.pkl', 'rb') as f:
   corpus_lg = pickle.load(f)

#### LDA Model in Gensim
Note that running eval_every=1 does this in batches of chunksize batches. The 20 is chosen from a HDP Process Test on the same data in another notebook

In [15]:
import logging
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train4 = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus4,
                           num_topics=20,
                           id2word=train_id2word4,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train4.save('lda_train4.model')

#### Coherence Model Score seems useless but included here

In [16]:
print(train_id2word4)
coherence_model_lda = CoherenceModel(model=lda_train4, texts=bigram_train4, dictionary=train_id2word4, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
coherence_lda

Dictionary(5378 unique tokens: ['pal', 'well', 'ft', 'normal', 'size']...)


0.25271502714345184

#### Example of Topics

In [17]:
lda_train4.print_topics(20,num_words=15)[:10]

[(0,
  '0.215*"love" + 0.054*"work" + 0.039*"something" + 0.031*"maybe" + 0.030*"done" + 0.030*"nice" + 0.028*"cause" + 0.027*"much" + 0.025*"keep" + 0.022*"making" + 0.021*"old" + 0.021*"true" + 0.021*"school" + 0.021*"guy" + 0.019*"night"'),
 (1,
  '0.095*"need" + 0.087*"thank" + 0.071*"omg" + 0.051*"take" + 0.046*"wanna" + 0.045*"girl" + 0.035*"okay" + 0.034*"better" + 0.034*"cant" + 0.034*"give" + 0.031*"days" + 0.028*"away" + 0.022*"use" + 0.021*"name" + 0.020*"hit"'),
 (2,
  '0.046*"stop" + 0.046*"thought" + 0.040*"youre" + 0.037*"years" + 0.036*"face" + 0.036*"find" + 0.034*"literally" + 0.032*"end" + 0.032*"bts" + 0.031*"soon" + 0.028*"didnt" + 0.025*"mind" + 0.023*"yall" + 0.019*"lot" + 0.018*"watched"'),
 (3,
  '0.191*"one" + 0.055*"also" + 0.038*"fuck" + 0.036*"things" + 0.035*"long" + 0.030*"fucking" + 0.028*"god" + 0.023*"left" + 0.021*"hes" + 0.021*"day" + 0.020*"took" + 0.018*"called" + 0.017*"open" + 0.017*"eyes" + 0.017*"awesome"'),
 (4,
  '0.078*"please" + 0.050*"alre

## Make Vectors

Now that we have an LDA model, we need to run all the reviews through it using 'get document topics'. A list comprehension on that output (2nd line in loop) will give the probability distribution of the topics for a specific review, and that's our feature vector. The other 2 lines in the loop are hand-engineered features.

In [18]:
train_vecs = []
for i in range(len(train_df)):
    top_topics = lda_train4.get_document_topics(train_corpus4[i], minimum_probability=0.0)
    #print(train_corpus4[i])
    topic_vec = [top_topics[i][1] for i in range(20)]
    #topic_vec.extend([train_df.iloc[i].real_counts]) # counts of reviews for restaurant
    #topic_vec.extend([len(train_df.iloc[i].text)]) # length review
    train_vecs.append(topic_vec)

In [47]:
print(train_vecs[0])
print(train_vecs[1])
print(train_vecs[2])

[0.016667739, 0.016667739, 0.016667739, 0.34996042, 0.016667739, 0.016667739, 0.016667739, 0.016667739, 0.016667739, 0.016667739, 0.016667739, 0.016667739, 0.016667739, 0.016667739, 0.016667739, 0.016667739, 0.016667739, 0.016667739, 0.35002026, 0.016667739]
[0.010000642, 0.4099947, 0.010000642, 0.010000642, 0.010000642, 0.010000642, 0.010000642, 0.010000642, 0.010000642, 0.010000642, 0.2100086, 0.010000642, 0.010000642, 0.010000642, 0.010000642, 0.010000642, 0.010000642, 0.010000642, 0.010000642, 0.2099858]
[0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.02500058, 0.52498895, 0.02500058]


#### Sklearn needs np arrays for CV-folds

In [48]:
X = np.array(train_vecs)

In [49]:
#y = np.array(train_df["".target)
#print(train_df['anger'])
y = train_df.loc[:,['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise'],] 
print(y)

       anger  disgust  fear  happiness  sadness  surprise
0          0        0     0          1        0         0
1          0        0     0          1        0         0
2          0        0     0          1        0         0
3          0        0     0          1        0         0
4          0        0     0          1        0         0
...      ...      ...   ...        ...      ...       ...
75369      0        0     0          0        1         0
75370      0        0     0          0        1         0
75371      0        0     0          0        0         1
75372      0        0     0          0        0         1
75373      0        0     0          0        0         0

[75374 rows x 6 columns]


In [50]:
with open('y.pkl', 'wb') as f:
    pickle.dump(y, f)

In [51]:
with open('X.pkl', 'wb') as f:
    pickle.dump(X, f)

## Run SVM classifier to predict emotion labels

In [53]:
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

def multiClassSVM(vectors, df):
    categories = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']
    X_train, X_test, y_train, y_test= train_test_split(vectors, df, random_state=42, test_size=0.33, shuffle=True)
    #X_train = train.vectors
    #X_test = test.vectors
    print(X_train.shape)
    print(X_test.shape)
    SVC_pipeline = Pipeline([
                    #('tfidf', TfidfVectorizer(stop_words=stop_words)),
                    ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
                ])
    accuracies = []
    predictions = []
    labels = []
    for category in categories:
        print('... Processing {}'.format(category))
        # train the model using X_dtm & y
        #print(X_train)
        SVC_pipeline.fit(X_train, y_train[category])
        # compute the testing accuracy
        prediction = SVC_pipeline.predict(X_test)
        #accuracy = accuracy_score(y_test[category], prediction)
        accuracy = f1_score(y_true=y_test[category], y_pred=prediction, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')
        predictions.extend(prediction)
        labels.extend(y_test[category])
        accuracies.append(accuracy)
        #print('Test accuracy is {}'.format(accuracy))
    accuracy = f1_score(y_true=labels, y_pred=predictions, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')
    return accuracy

In [54]:
accuracy = multiClassSVM(X, train_df)
print(accuracy)

(50500, 20)
(24874, 20)
... Processing anger
... Processing disgust
... Processing fear
... Processing happiness
... Processing sadness
... Processing surprise
0.24282895735316434


#### Final 5-fold CV loop for training on 2016 review data

In [85]:
kf = KFold(5, shuffle=True, random_state=42)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []
#print(kf.split(X, y))
for train_ind, val_ind in kf.split(X, y):
    # Assign CV IDX
    print(train_ind)
    #print(val_ind)
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # Logistic Regression Mini-Batch SGD
    sgd = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # SGD Modified Huber
    sgd_huber = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average='binary'))

print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')

[   1    2    4 ... 9892 9893 9895]


KeyError: "None of [Int64Index([   1,    2,    4,    5,    6,    7,    9,   11,   13,   15,\n            ...\n            9882, 9883, 9884, 9887, 9888, 9890, 9891, 9892, 9893, 9895],\n           dtype='int64', length=7916)] are in the [columns]"